<a href="https://colab.research.google.com/github/emidiob/ML_Art_Resources/blob/main/MidasV3Large_V3Hybrid_V21small_Pythorch_ImageSequence_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**If you find this notebooks helpful, follow me on [Instagram](https://www.instagram.com/emidiobattipaglia/) and [Twitter](https://twitter.com/Emidio_B)**

# ⭕ Check Hardware

In [ ]:
print('CPU:') 
!lscpu |grep 'Model name'
print('GPU:') 
!nvidia-smi -q
!nvidia-smi
print('Memory:') 
!free -h --si | awk  '/Mem:/{print $2}'
print('Memory Info:') 
!cat /proc/meminfo

CPU:
Model name:          Intel(R) Xeon(R) CPU @ 2.30GHz
GPU:

==============NVSMI LOG==============

Timestamp                                 : Mon Dec 27 07:36:28 2021
Driver Version                            : 460.32.03
CUDA Version                              : 11.2

Attached GPUs                             : 1
GPU 00000000:00:04.0
    Product Name                          : Tesla P100-PCIE-16GB
    Product Brand                         : Tesla
    Product Architecture                  : Pascal
    Display Mode                          : Enabled
    Display Active                        : Disabled
    Persistence Mode                      : Disabled
    MIG Mode
        Current                           : N/A
        Pending                           : N/A
    Accounting Mode                       : Disabled
    Accounting Mode Buffer Size           : 4000
    Driver Model
        Current                           : N/A
        Pending                           : N/A
    Serial

# ⭕ Monta Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#⭕ Sposta archivio nella directory principale e Unzippa i files


In [ ]:
!cp /content/drive/MyDrive/midas/input.zip /content/

!pip install patool
import patoolib
patoolib.extract_archive("/content/input.zip",outdir="/content/")

!rm /content/input.zip



# Code

Download an image from the PyTorch homepage
Load a model (see https://github.com/intel-isl/MiDaS/#Accuracy for an overview)

In [ ]:
!pip install timm

In [ ]:
import cv2
import torch
import urllib.request

import matplotlib.pyplot as plt

import os

path = "/content/export"

try:
    os.mkdir(path)
except OSError:
    print ("Creation of the directory %s failed" % path)
else:
    print ("Successfully created the directory %s " % path)

#model_type = "DPT_Large"     # MiDaS v3 - Large     (highest accuracy, slowest inference speed)
#model_type = "DPT_Hybrid"   # MiDaS v3 - Hybrid    (medium accuracy, medium inference speed)
model_type = "MiDaS_small"  # MiDaS v2.1 - Small   (lowest accuracy, highest inference speed)

midas = torch.hub.load("intel-isl/MiDaS", model_type)

#Move to GPU
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
midas.to(device)
midas.eval()

#Load Transforms and resize to normalize the image for large or small models
midas_transforms = torch.hub.load("intel-isl/MiDaS", "transforms")

if model_type == "DPT_Large" or model_type == "DPT_Hybrid":
    transform = midas_transforms.dpt_transform
else:
    transform = midas_transforms.small_transform

Load image and apply transforms

In [ ]:
directory = r'/content/input'
for entry in os.scandir(directory):
    if (entry.path.endswith(".jpg")
            or entry.path.endswith(".png")
                    or entry.path.endswith(".jpeg")) and entry.is_file():
        
        print("Reading image", entry.name)
        
        img = cv2.imread(entry.path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        input_batch = transform(img).to(device)

        with torch.no_grad():
            prediction = midas(input_batch)

            prediction = torch.nn.functional.interpolate(
                prediction.unsqueeze(1),
                size=img.shape[:2],
                mode="bicubic",
                align_corners=False,
            ).squeeze()


        output = prediction.cpu().numpy()
        print("Writing image", entry.name)

        cv2.imwrite(f'/content/export/{entry.name}',output)

!zip -r /content/export.zip /content/export
print("Folder Zipped")

!cp /content/export.zip /content/drive/MyDrive/midas/
print("Folder Copied to destination")